In [1]:
import os
import pandas as pd
from kafka import KafkaProducer

In [5]:
#variables globales a ser usadas por el programa
DATA_INPUT_FILES = ["../data/input/customer.csv","../data/input/date.csv","../data/input/product.csv","../data/input/reseller.csv","../data/input/sales.csv","../data/input/sales_order.csv","../data/input/sales_territory.csv"]
VM_PUBLIC_IP = ""
KAFKA_SERVER_IP = f"{VM_PUBLIC_IP}:9092"
PROJECTS_NAME = "prueba"
KAFKA_TOPIC_NAME = [f"{PROJECTS_NAME}.customer",f"{PROJECTS_NAME}.date",f"{PROJECTS_NAME}.product",f"{PROJECTS_NAME}.reseller",f"{PROJECTS_NAME}.sales",f"{PROJECTS_NAME}.sales-order",f"{PROJECTS_NAME}.sales-territory"]

In [ ]:
#inicializo la conexion con el productor de kafka
kafka_producer = KafkaProducer(bootstrap_servers=KAFKA_SERVER_IP)

In [16]:
def data_2_kafka(input_data_path:str, kafka_topic_name:str) -> int:
    """Leer una url de datos de entrada los cuales deben de ser CSV y enviarlos al topico de kafka

    Args:
        input_data_path (str): _description_
        kafka_topic_name (str): _description_

    Returns:
        int: _description_
    """
    try:
        #me aseguro de que el archivo si existe
        assert os.path.exists(input_data_path), "El path al archivo de datos de entrada es erroneo, o el archivo no existe."
        file = os.path.splitext(input_data_path)
        assert file[1] == ".csv", f"El archivo {file[0]} debe de ser de extension CSV."
        df = pd.read_csv(input_data_path)
        json_message = df.to_json(orient="records") #cada fila del df se convierte en un JSON independiente
        #kafka_producer.send(kafka_topic_name, json_message.encode('utf-8'))
        return 0
    except Exception as error:
        print(error)
        return 1

In [14]:
def main():
    assert len(DATA_INPUT_FILES) == len(KAFKA_TOPIC_NAME), "Deben de haber tantos topicos como archivos de entrada hallan."
    for data_file, kafka_topic in zip(DATA_INPUT_FILES,KAFKA_TOPIC_NAME):
        data_2_kafka(
            input_data_path=data_file,
            kafka_topic_name=kafka_topic,
        )

In [ ]:
if __name__ == "__main__":
    main()